In [ ]:
!pip3 install tensorflow_text>=2.0.0rc0

In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text
import pandas as pd

In [ ]:
data = pd.read_csv("CustFlowDataset_full_red_mcc.csv", index_col=None)

In [ ]:
data

,Unnamed: 0,0,10,11,12,13,16,17,19,2,20,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,5,51,52,...,85,86,AV_01,AV_09,AV_mean,AV_med,F_01,F_09,F_mean,F_med,MCC87,av_flow,count,high_price,low_price,mean_tr,med_tr,mid_price,nei_01_mean,nei_mean_90,nei_tr_mean,same_F,same_av_flow,same_high,same_low,same_mid,same_size,same_tr,sz,sz_mean,time_to_open,tr_01,tr_90,org_name,category1,category2,category3,category1_mcc_string,category2_mcc_string,category3_mcc_string
0,""" Kafe_Berloga"", G SANKT-PETER, RU 40000000000...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1.897050,3.704855,2.738078,2.612331,835.5,1814.5,1338.333333,1365.0,63,1.769231,115,0,0,242.739130,215.0,1,69.000000,732.250000,309.596665,1472.000000,2.511945,0,1,0,1,133.220788,7,7,15.533333,70.0,430.0,Берлога,Быстрое питание,none,none,fast food,none,none
1,"""10 LINIYA V.O."", SANKT-PETERBU, RU 22335954",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,...,0,0,1.407761,26.924689,10.258272,6.352201,846.0,13143.4,5749.360000,3552.0,63,2.020376,1289,0,0,380.182118,260.0,1,195.084000,4147.110400,1526.279665,3177.166667,5.589412,2,2,2,6,486.148637,38,38,0.000000,119.0,828.0,Линия,Хостел,none,none,hotels,none,none
2,"""1000 MELOCHEI"" SHOP, SANKT-PETERBU, RU 709736...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1.417736,37.210031,10.519003,2.653543,208.6,23740.0,6243.777778,1092.0,60,0.932203,110,0,0,305.281818,190.0,1,104.625556,1165.594444,519.822811,8799.666667,14.749593,3,2,1,6,359.472502,10,10,17.300000,50.0,570.0,1000 Мелочей,Товары для дома,Магазин хозтоваров и бытовой химии,Магазин канцтоваров,malls,malls,malls
3,"""13 RULES"", G SANKT-PETER, RU 400000000000024",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,1.720886,6.434223,5.121963,3.462500,223.3,3499.1,2253.333333,1196.5,5,1.324786,310,0,0,441.896065,350.0,1,102.749167,856.182500,383.551299,424.000000,0.683871,0,0,1,1,690.271226,18,18,13.466667,195.0,765.0,R-13.ru,Интернет-магазин,Магазин автозапчастей и автотоваров,none,software,auto,none
4,"""1703"", SANKT-PETERBU, RU 4935404002",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1.201942,10.718724,5.116683,3.007560,382.3,6837.3,3063.000000,1696.0,63,1.469136,119,0,0,402.100840,300.0,1,122.250000,501.750000,282.039649,3202.333333,5.438152,0,0,3,3,321.389224,4,4,18.600000,250.0,700.0,1703,"Бар, паб",Ночной клуб,none,bars,bars,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4775,"PAPER SHOP, SANKT-PETERBU, RU 550000108252",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,0,0,0,0,0,0,0,...,0,0,0.443431,2.288459,1.373593,1.036176,185.0,1126.0,626.638889,405.5,45,1.006309,638,1,0,4140.446708,2896.0,0,1511.436111,11490.269444,5506.379144,616.785714,1.349221,22,0,6,28,6280.393137,37,37,0.100000,859.0,8796.0,Roll-Paper,Производство и продажа бумаги,Материалы для полиграфии,Расходные материалы для оргтехники,fallback services,printing services,fallback services
4776,"PIONER, SANKT-PETERBU, RU 551000021672",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.718919,43.983752,15.511811,1.832763,228.5,13812.0,5068.333333,1164.5,60,0.265823,42,1,0,417.428571,314.0,0,73.965000,934.126667,422.673003,9220.000000,29.422501,2,0,1,3,489.030608,7,7,15.733333,144.0,854.0,Pioner-Gun.ru,Интернет-магазин,none,none,software,none,none
4777,"PROSVESHCHENIE, SANKT-PETERBU, RU 550000040636",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,2.153135,30.804075,16.575627,16.818182,1373.7,19653.0,10575.250000,10730.0,

In [ ]:
data = data[['Unnamed: 0', 'tr_01', 'mean_tr', 'med_tr', 'tr_90', 'av_flow', 
             'category1', 'category2', 'category3', "category1_mcc_string", 
             "category2_mcc_string", "category3_mcc_string"]].fillna("none")
data['category1'] = data['category1'].str.lower()
data['category2'] = data['category2'].str.lower()
data['category3'] = data['category3'].str.lower()
# data = data[data['category1'] != "none"]

In [ ]:
data

,Unnamed: 0,tr_01,mean_tr,med_tr,tr_90,av_flow,category1,category2,category3,category1_mcc_string,category2_mcc_string,category3_mcc_string
0,""" Kafe_Berloga"", G SANKT-PETER, RU 40000000000...",70.0,242.739130,215.0,430.0,1.769231,быстрое питание,none,none,fast food,none,none
1,"""10 LINIYA V.O."", SANKT-PETERBU, RU 22335954",119.0,380.182118,260.0,828.0,2.020376,хостел,none,none,hotels,none,none
2,"""1000 MELOCHEI"" SHOP, SANKT-PETERBU, RU 709736...",50.0,305.281818,190.0,570.0,0.932203,товары для дома,магазин хозтоваров и бытовой химии,магазин канцтоваров,malls,malls,malls
3,"""13 RULES"", G SANKT-PETER, RU 400000000000024",195.0,441.896065,350.0,765.0,1.324786,интернет-магазин,магазин автозапчастей и автотоваров,none,software,auto,none
4,"""1703"", SANKT-PETERBU, RU 4935404002",250.0,402.100840,300.0,700.0,1.469136,"бар, паб",ночной клуб,none,bars,bars,none
...,...,...,...,...,...,...,...,...,...,...,...,...
4775,"PAPER SHOP, SANKT-PETERBU, RU 550000108252",859.0,4140.446708,2896.0,8796.0,1.006309,производство и продажа бумаги,материалы для полиграфии,расходные материалы для оргтехники,fallback services,printing services,fallback services
4776,"PIONER, SANKT-PETERBU, RU 551000021672",144.0,417.428571,314.0,854.0,0.265823,интернет-магазин,none,none,software,none,none
4777,"PROSVESHCHENIE, SANKT-PETERBU, RU 550000040636",127.0,314.452381,280.0,540.0,0.708861,издательские услуги,энергоснабжение,сантехнические работы,office service,none,none
4778,"SUPERMARKET SPAR, SANKT-PETERBU, RU 550000047987",91.8,500.521955,360.0,1247.5,14.070397,супермаркет,магазин продуктов,none,supermarket,supermarket,none


In [ ]:
tr_stat = data[['tr_01', 'mean_tr', 'med_tr', 'tr_90']].values

def tr_stat_apply(row):
  return tr_stat[row.name, :]

data['tr_stat'] = data.apply(tr_stat_apply, raw=False, axis=1)
data = data[['Unnamed: 0', 'tr_stat', 'av_flow', 'category1', 
             'category2', 'category3', "category1_mcc_string", 
             "category2_mcc_string", "category3_mcc_string"]]

data = data[data['category1'] != 'none']
data.to_csv("CustFlowDataset_final_1", index=None)

In [ ]:
data

,Unnamed: 0,tr_stat,av_flow,category1,category2,category3,category1_mcc_string,category2_mcc_string,category3_mcc_string
0,""" Kafe_Berloga"", G SANKT-PETER, RU 40000000000...","[70.0, 242.7391304348, 215.0, 430.0]",1.769231,быстрое питание,none,none,fast food,none,none
1,"""10 LINIYA V.O."", SANKT-PETERBU, RU 22335954","[119.0, 380.18211792089994, 260.0, 828.0]",2.020376,хостел,none,none,hotels,none,none
2,"""1000 MELOCHEI"" SHOP, SANKT-PETERBU, RU 709736...","[50.0, 305.2818181818, 190.0, 570.0]",0.932203,товары для дома,магазин хозтоваров и бытовой химии,магазин канцтоваров,malls,malls,malls
3,"""13 RULES"", G SANKT-PETER, RU 400000000000024","[195.0, 441.8960645161, 350.0, 765.0]",1.324786,интернет-магазин,магазин автозапчастей и автотоваров,none,software,auto,none
4,"""1703"", SANKT-PETERBU, RU 4935404002","[250.0, 402.1008403361, 300.0, 700.0]",1.469136,"бар, паб",ночной клуб,none,bars,bars,none
...,...,...,...,...,...,...,...,...,...
4775,"PAPER SHOP, SANKT-PETERBU, RU 550000108252","[859.0, 4140.446708464, 2896.0, 8796.0]",1.006309,производство и продажа бумаги,материалы для полиграфии,расходные материалы для оргтехники,fallback services,printing services,fallback services
4776,"PIONER, SANKT-PETERBU, RU 551000021672","[144.0, 417.4285714286, 314.0, 854.0]",0.265823,интернет-магазин,none,none,software,none,none
4777,"PROSVESHCHENIE, SANKT-PETERBU, RU 550000040636","[127.0, 314.4523809524, 280.0, 540.0]",0.708861,издательские услуги,энергоснабжение,сантехнические работы,office service,none,none
4778,"SUPERMARKET SPAR, SANKT-PETERBU, RU 550000047987","[91.8, 500.52195509940003, 360.0, 1247.5]",14.070397,супермаркет,магазин продуктов,none,supermarket,supermarket,none


In [ ]:
names = data['Unnamed: 0'].values

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
embedded_names = embed(names[0:780]).numpy()

In [ ]:
embedded_names

array([[-0.03274743, -0.01375353,  0.06619928, ..., -0.00170163,
        -0.0815651 ,  0.06318818],
       [-0.04035876,  0.00328976,  0.04160535, ..., -0.04766767,
        -0.0554484 ,  0.08389492],
       [ 0.03898586, -0.02824285,  0.05979111, ..., -0.03552209,
        -0.04620165,  0.02017843],
       ...,
       [-0.07201201, -0.03540159,  0.07708828, ..., -0.02639846,
         0.0065521 ,  0.09524035],
       [-0.07460461, -0.00518145,  0.10674159, ..., -0.02622633,
        -0.01375214,  0.08557922],
       [-0.07564533,  0.05280367,  0.07018242, ..., -0.0635398 ,
        -0.00026263,  0.0324561 ]], dtype=float32)

In [ ]:
for i in range(1, 5):
  embedded_temp = embed(names[780 + (i - 1) * 1000: 780 + i * 1000]).numpy()
  embedded_names = np.concatenate((embedded_names, embedded_temp), axis=0)

In [ ]:
names_data = pd.DataFrame(data=names, columns=['Unnamed: 0'])

In [ ]:
names_data

,Unnamed: 0
0,""" Kafe_Berloga"", G SANKT-PETER, RU 40000000000..."
1,"""10 LINIYA V.O."", SANKT-PETERBU, RU 22335954"
2,"""1000 MELOCHEI"" SHOP, SANKT-PETERBU, RU 709736..."
3,"""13 RULES"", G SANKT-PETER, RU 400000000000024"
4,"""1703"", SANKT-PETERBU, RU 4935404002"
...,...
4775,"PAPER SHOP, SANKT-PETERBU, RU 550000108252"
4776,"PIONER, SANKT-PETERBU, RU 551000021672"
4777,"PROSVESHCHENIE, SANKT-PETERBU, RU 550000040636"
4778,"SUPERMARKET SPAR, SANKT-PETERBU, RU 550000047987"


In [ ]:
def test(row):
  return embedded_names[row.name, :]

names_data['org_name_embedded'] = names_data.apply(test, raw=False, axis=1)

In [ ]:
data = pd.merge(data, names_data, on='Unnamed: 0')
data = data[['Unnamed: 0', 'org_name_embedded', 'tr_stat', 'av_flow', 
             'category1', 'category2', 'category3', "category1_mcc_string", 
             "category2_mcc_string", "category3_mcc_string"]]

In [ ]:
data

,Unnamed: 0,org_name_embedded,tr_stat,av_flow,category1,category2,category3,category1_mcc_string,category2_mcc_string,category3_mcc_string
0,""" Kafe_Berloga"", G SANKT-PETER, RU 40000000000...","[-0.032747433, -0.013753531, 0.06619928, -0.01...","[70.0, 242.7391304348, 215.0, 430.0]",1.769231,быстрое питание,none,none,fast food,none,none
1,"""10 LINIYA V.O."", SANKT-PETERBU, RU 22335954","[-0.040358763, 0.0032897622, 0.041605353, 0.00...","[119.0, 380.18211792089994, 260.0, 828.0]",2.020376,хостел,none,none,hotels,none,none
2,"""1000 MELOCHEI"" SHOP, SANKT-PETERBU, RU 709736...","[0.038985863, -0.028242854, 0.05979111, -0.029...","[50.0, 305.2818181818, 190.0, 570.0]",0.932203,товары для дома,магазин хозтоваров и бытовой химии,магазин канцтоваров,malls,malls,malls
3,"""13 RULES"", G SANKT-PETER, RU 400000000000024","[0.04536989, -0.016444983, 0.0673368, -0.03159...","[195.0, 441.8960645161, 350.0, 765.0]",1.324786,интернет-магазин,магазин автозапчастей и автотоваров,none,software,auto,none
4,"""1703"", SANKT-PETERBU, RU 4935404002","[-0.09249232, -0.003072308, 0.06466564, 0.0082...","[250.0, 402.1008403361, 300.0, 700.0]",1.469136,"бар, паб",ночной клуб,none,bars,bars,none
...,...,...,...,...,...,...,...,...,...,...
4775,"PAPER SHOP, SANKT-PETERBU, RU 550000108252","[-0.03880448, 0.01119087, 0.04527383, -0.00338...","[859.0, 4140.446708464, 2896.0, 8796.0]",1.006309,производство и продажа бумаги,материалы для полиграфии,расходные материалы для оргтехники,fallback services,printing services,fallback services
4776,"PIONER, SANKT-PETERBU, RU 551000021672","[-0.10323684, 0.03639785, 0.07304671, 0.005899...","[144.0, 417.4285714286, 314.0, 854.0]",0.265823,интернет-магазин,none,none,software,none,none
4777,"PROSVESHCHENIE, SANKT-PETERBU, RU 550000040636","[-0.097637825, -0.008753444, 0.078081645, -0.0...","[127.0, 314.4523809524, 280.0, 540.0]",0.708861,издательские услуги,энергоснабжение,сантехнические работы,office service,none,none
4778,"SUPERMARKET SPAR, SANKT-PETERBU, RU 550000047987","[-0.024029909, 0.014203973, -0.017981742, -0.0...","[91.8, 500.52195509940003, 360.0, 1247.5]",14.070397,супермаркет,магазин продуктов,none,supermarket,supermarket,none


In [ ]:
data.to_csv("CFD_final.csv", index=False)